# Kaggle Santander competition

In [23]:
# general & data analysis imports
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc

import lightgbm as lgb





## data load

In [2]:
train_dataset=pd.read_csv('train.csv')
test_dataset=pd.read_csv('test.csv')

## data analysis

In [3]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 202 entries, ID_code to var_199
dtypes: float64(200), int64(1), object(1)
memory usage: 308.2+ MB


In [ ]:
train_dataset.describe()

In [ ]:
train_dataset.columns

In [ ]:
train_dataset.head()

In [ ]:
train_dataset.isnull().sum().sum()

## data preprocessing

In [4]:
# remove dependent variable from train set to have the same structure as test set
df_target=train_dataset['target'].copy()
df_train=train_dataset.drop(['ID_code','target'], axis=1)
df_test=test_dataset.drop('ID_code', axis=1)

In [5]:
X_train,X_val,y_train,y_val=train_test_split(df_train.values,df_target.values,test_size=0.15,random_state=1, shuffle=True)
X_test=df_test.values

In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)

(170000, 200)
(170000,)
(30000, 200)
(30000,)
(200000, 200)


## training ad evaluation

In [36]:
param = {'objective': 'binary', 'metric': 'auc', 'learning_rate': 0.01, 'num_rounds': 2000, 'verbose': 1}
#test_pred = np.zeros(len(test_data))
train_set = lgb.Dataset(X_train, label=y_train)
val_set = lgb.Dataset(X_val, label=y_val)
clf = lgb.train(param, train_set, valid_sets=[train_set,val_set], verbose_eval=200)



/home/tslow/miniconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[200]	training's auc: 0.845825	valid_1's auc: 0.806869
[400]	training's auc: 0.885131	valid_1's auc: 0.839273
[600]	training's auc: 0.906515	valid_1's auc: 0.855849
[800]	training's auc: 0.920279	valid_1's auc: 0.865749
[1000]	training's auc: 0.930087	valid_1's auc: 0.87252
[1200]	training's auc: 0.93768	valid_1's auc: 0.877353
[1400]	training's auc: 0.943898	valid_1's auc: 0.881115
[1600]	training's auc: 0.949159	valid_1's auc: 0.884124
[1800]	training's auc: 0.953687	valid_1's auc: 0.886426
[2000]	training's auc: 0.957638	valid_1's auc: 0.888173


In [34]:
y_val_probs=clf.predict(X_val)
y_val_preds=np.where(y_val_probs>0.5,1,0)

In [35]:
roc_auc_score(y_val, y_val_probs)

0.888173465746087

In [12]:
y_probs=clf.predict(X_test)

In [13]:
y_preds=np.where(y_probs>0.5,1,0)

In [24]:
auc(y_true, y_preds)

NameError: name 'y_true' is not defined

## submision

In [18]:
submission_df = pd.DataFrame({"ID_code":test_dataset["ID_code"].values})
submission_df["target"] = y_probs
submission_df.to_csv("submission.csv", index=False)